In [1]:
import pathlib


In [9]:
config = "../config/debsug.yaml"
config_path = pathlib.Path(config)

if config_path.exists():
    print(config_path)
else:
    raise Fil

FileNotFoundError: Config file does not exist